# Ассоциативные правила
Скрипт позволяет реализовать алгоритм поиска ассоциативных правил


In [1]:
# импортируем библиотеки 
import pandas as pd
import numpy as np

In [2]:
#загружаем данные их файла MS Excel
sales=pd.read_excel('DataSet.xlsx')
#проверяем результат, выводим первые пять строчек кода
sales.head()

,Дата заказа,Артикул,Город,Ассортимент,Количество,Сумма,Прибыль,Номер заказа,Клиент
0,2019-09-01,1000006,Москва,Apple iPhone X,10,99000,990000,26833563,Client_1022
1,2019-09-02,1000001,Екатеринбург,Apple iPad Pro,5,99000,495000,26833563,Client_1023
2,2019-09-03,1000007,Санкт-Петербург,Apple iPad Mini 13,6,99000,594000,26833563,Client_1029
3,2019-09-04,1000005,Москва,Apple Watch,8,99000,792000,26833563,Client_1028
4,2019-09-05,1000006,Москва,Apple iPhone X,4,99000,396000,26833563,Client_1029


In [3]:
#преобразуем таблицу таким образом, чтобы столбцы были наименования ассортимента, строки клиенты, а на пересечении
# на какую сумму они купили данных товаров
# фактически для построение данной матрицы нужны столбцы имя клиента или чек, название позции, сумма или количество купленных товаров
UID_sales=sales.pivot_table(values='Сумма', columns=['Ассортимент'], index='Клиент', aggfunc=np.sum)
#Вместо NaN (значение отсутствует) проставим в ДатаФрейм 0-ки:
UID_sales[np.isnan(UID_sales)] = 0
# все значения больше 0 заменим 1, чтобы больше проявить правила. Делать не обязательно, но это повышает точность
UID_sales[UID_sales>0]=1
# пять первых строк новой таблицы
UID_sales.head()

Ассортимент,Apple Mac Book Air,Apple Mac Book Pro,Apple Watch,Apple iMac 27,Apple iPad Mini 100,Apple iPad Mini 101,Apple iPad Mini 102,Apple iPad Mini 13,Apple iPad Mini 14,Apple iPad Mini 15,...,Apple iPad Mini 92,Apple iPad Mini 93,Apple iPad Mini 94,Apple iPad Mini 95,Apple iPad Mini 96,Apple iPad Mini 97,Apple iPad Mini 98,Apple iPad Mini 99,Apple iPad Pro,Apple iPhone X
Клиент,,,,,,,,,,,,,,,,,,,,,
Client_1021,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Client_1022,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Client_1023,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Client_1024,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Client_1025,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [4]:
#Создадим вначале список транзакций в формате [[1,4,5], [3,5,3], ....] - то есть каждый элемент списка - 
#список того, что купил клиент:

#Создадим функцию для создания списка транзакций:
def transaction_list(df):
    list_external=[]
    for i in range(df.shape[0]):
        list_internal=[]
        data=df.iloc[i]
        index=data[data>0]
        for element in index.index:
            list_internal.append(element)
        list_external.append(list_internal)
    return list_external
# используем функцию, чтобы преобразовать массив в список транзакций
transactions=transaction_list(UID_sales[:1000])
# это список товаров, которые содержаться в чеке или их покупает конкретный клиент
transactions[1]

['Apple Mac Book Pro',
 'Apple Watch',
 'Apple iPad Mini 13',
 'Apple iPad Pro',
 'Apple iPhone X']

Следующая ячейка устанавливает пакет pymining, которого нет по умолчанию в данной сборке. 


In [5]:
 pip install pymining

In [6]:
# импортируем дополнительные библиотеки
from pymining import itemmining, assocrules, perftesting
#подготовливаем список стандартными функциями
relim_input = itemmining.get_relim_input(transactions)
item_sets = itemmining.relim(relim_input, min_support=1)
#item_sets

In [7]:
# запускаем расчет ассоциативным правил
# внимание, локальная система может не справиться
#min_support=5 - минимум 5 реализаций правила, min_confidence=0.6 - минимальный порог вероятности
rules = assocrules.mine_assoc_rules(item_sets, min_support=5, min_confidence=0.6)
rules

[(frozenset({'Apple Mac Book Air', 'Apple Watch', 'Apple iPad Pro'}),
  frozenset({'Apple iPad Mini 13'}),
  5,
  1.0),
 (frozenset({'Apple Watch', 'Apple iPad Pro'}),
  frozenset({'Apple Mac Book Air', 'Apple iPad Mini 13'}),
  5,
  0.8333333333333334),
 (frozenset({'Apple Watch'}),
  frozenset({'Apple Mac Book Air', 'Apple iPad Mini 13', 'Apple iPad Pro'}),
  5,
  0.8333333333333334),
 (frozenset({'Apple iPad Pro'}),
  frozenset({'Apple Mac Book Air', 'Apple Watch', 'Apple iPad Mini 13'}),
  5,
  0.625),
 (frozenset({'Apple Mac Book Air', 'Apple Watch'}),
  frozenset({'Apple iPad Mini 13', 'Apple iPad Pro'}),
  5,
  1.0),
 (frozenset({'Apple Mac Book Air'}),
  frozenset({'Apple Watch', 'Apple iPad Mini 13', 'Apple iPad Pro'}),
  5,
  0.7142857142857143),
 (frozenset({'Apple Mac Book Air', 'Apple iPad Pro'}),
  frozenset({'Apple Watch', 'Apple iPad Mini 13'}),
  5,
  0.7142857142857143),
 (frozenset({'Apple Watch', 'Apple iPad Mini 13', 'Apple iPad Pro'}),
  frozenset({'Apple Mac Book

In [8]:
#Создадим функцию для записи ассоциативных правил:
def write_rules2(rul):
    retMass=[]
    for el in rul:
        basis=''
        for iterator in iter(el[0]):
            basis=basis+iterator+'-'
        conclusion=''
        for iterator in iter(el[1]):
            conclusion=conclusion+iterator+'-'    
        retMass.append([basis, conclusion, str(el[2]), str(el[3])])
    return retMass

In [9]:
#созраним правила
rul1=write_rules2(rules)
#преобразуем выходной массив в DataFrame и сохраним в Excel
df_rules=pd.DataFrame(rul1, columns=('Посыл', 'Следствие', 'Поддержка', 'Достоверность'))
#имя файла AssocRules.xlsx
writer = pd.ExcelWriter('AssResult.xlsx')
df_rules.to_excel(writer,'AR')
writer.save()